In [2]:
import os
%pwd

'/home/towet/Desktop/OpenProjects/Alzheimer-MRI-Classification-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/OpenProjects/Alzheimer-MRI-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    dataset: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [6]:
from AlzheimerClassifier.constants import *
from AlzheimerClassifier.utils.common import read_yaml, create_directories, save_json

2024-02-09 10:31:59.751467: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-09 10:31:59.812611: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-02-09 10:31:59.812660: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-02-09 10:31:59.814264: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-02-09 10:31:59.824055: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-02-09 10:32:06.283407: W tensorflow/compiler/tf2tensorrt/utils/py_utils.

In [ ]:
MLFLOW_TRACKING_URI=https://dagshub.com/Towet-Tum/Alzheimer-MRI-Classification-Project.mlflow \
MLFLOW_TRACKING_USERNAME=Towet-Tum \
MLFLOW_TRACKING_PASSWORD=39c66009c3d7d6d659cbc691e65a93c46873d3c4 \
python script.py

In [7]:
class ConfigurationManager:
    def __init__(self, 
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        
        create_directories([self.config.artifacts_root])
    
    def get_evaluation_config(self) -> EvaluationConfig:
        
        dataset = os.path.join("artifacts", "data_ingestion", "dataset", "AugmentedAlzheimerDataset")
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.Xception.keras",
            mlflow_uri="https://dagshub.com/Towet-Tum/Alzheimer-MRI-Classification-Project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMG_SIZE,
            params_batch_size=self.params.BATCH_SIZE,
            dataset=Path(dataset)
        )
        return eval_config

In [8]:
import tensorflow as tf
from pathlib import Path
import mlflow
import mlflow.keras
from urllib.parse import urlparse
from AlzheimerClassifier.utils.common import load_data, process_data

In [9]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config



    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        df = load_data(self.config.dataset)
        _, _, test_gen = process_data(df)
        self.model = self.load_model(self.config.path_of_model)
        ts_length = len(test_gen)
        test_batch_size = min(32, ts_length)
        test_steps = ts_length // test_batch_size
        self.score = self.model.evaluate(test_gen, steps=test_steps, verbose=1)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                mlflow.keras.log_model(self.model, "model", registered_model_name="XceptionModel")
            else:
                mlflow.keras.log_model(self.model, "model")

In [10]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

[2024-02-09 10:32:33,475: INFO: common: yaml file: config/config.yaml loaded successfully]


[2024-02-09 10:32:33,520: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-09 10:32:33,521: INFO: common: created directory at: artifacts]
Found 27187 validated image filenames belonging to 4 classes.
Found 3398 validated image filenames belonging to 4 classes.
Found 3399 validated image filenames belonging to 4 classes.


2024-02-09 10:32:34.635740: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-02-09 10:34:35.755944: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.
2024-02-09 10:34:35.959572: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.
2024-02-09 10:34:36.112302: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.


1/3 [=========>....................] - ETA: 20s - loss: 8.2503e-05 - accuracy: 1.0000

2024-02-09 10:34:44.097365: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.
2024-02-09 10:34:44.287853: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 194658304 exceeds 10% of free system memory.


3/3 [==============================] - 27s 8s/step - loss: 2.6234e-04 - accuracy: 1.0000
[2024-02-09 10:35:00,254: INFO: common: json file saved at: scores.json]


2024/02/09 10:35:00 WARNING mlflow.tensorflow: You are saving a TensorFlow Core model or Keras model without a signature. Inference with mlflow.pyfunc.spark_udf() will not work unless the model's pyfunc representation accepts pandas DataFrames as inference inputs.


[2024-02-09 10:35:19,194: INFO: builder_impl: Assets written to: /tmp/tmpaon21tfp/model/data/model/assets]
